In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from collections import Counter
import re
import warnings # Turn off warnings
warnings.filterwarnings('ignore')
# Pandas options
pd.set_option("max_rows", 30)
pd.set_option("max_columns", None)
pd.set_option("precision", 3)

In [2]:
def format_turnstile_df(turnstile_csv_df):
    """
    turnstile_csv_df is the dataframe of the imported turnstile.csv file
    Warning: Run once per file ONLY. The function will throw an error if run twice on the same file.
    """
    turnstile_df = turnstile_csv_df
    
    # Reformatting steps (Warning: May take a few seconds.)
    turnstile_df.columns = turnstile_df.columns.str.replace(' ','') # Remove spaces in column names
    turnstile_df.columns = turnstile_df.columns.str.replace('/','') # Remove "/" in column names
    turnstile_df['DATETIMERAW'] = "" # Create new column called 'DATETIMERAW'
    turnstile_df['DATETIMERAW'] = turnstile_df.DATE + turnstile_df.TIME # Populate 'DATETIMERAW' with date and time concatenated string
    turnstile_df.DATETIMERAW = turnstile_df.DATETIMERAW.apply(lambda x : dt.datetime.strptime(x, "%m/%d/%Y%H:%M:%S")) # Convert DATETIMERAW into datetime object
    turnstile_df.TIME = turnstile_df.TIME.apply(lambda x : dt.datetime.strptime(x, "%H:%M:%S")) # Convert TIME into datetime object
    turnstile_df = turnstile_df.drop(columns = ['DATE']) # Drop DATE column
    return turnstile_df

def valuecount(timestamp1, timestamp2):
    """
    This function references turnstile_df, which is the output of format_turnstile_df().
    It grabs the data at two individual time stamps and outputs the difference in the count between those timestamps.
    Both timestamp1 and timestamp2 should be datetime objects. Also, timestamp2 should be greater than timestamp1.
    This function is not run as a standalone function. It is run within the grabweek() function.
    """
    
    # Grab data
    turnstile_df1 = turnstile_df.loc[turnstile_df['DATETIMERAW'].isin([timestamp1])].reset_index(drop=True) # Grab dataset for timestamp1
    turnstile_df2 = turnstile_df.loc[turnstile_df['DATETIMERAW'].isin([timestamp2])].reset_index(drop=True) # Grab dataset for timestamp2
    
    # Merge two datasets, drop any rows that don't form a complete dataset
    turnstile_df_merge12 = turnstile_df1.merge(turnstile_df2.drop_duplicates(), on=["CA", "UNIT", "SCP", "STATION", "LINENAME", "DIVISION", "DESC"], how='outer').dropna().reset_index(drop=True)
    
    # Create a new dataframe with essential data
    turnstile_df_valuecounts = turnstile_df_merge12[["STATION"]] # Grabbing "STATION" name data
    turnstile_df_valuecounts["ENTRIES DIFFERENCE"] = (turnstile_df_merge12['ENTRIES_x'] - turnstile_df_merge12['ENTRIES_y']).abs() # Grabbing "ENTRIES" difference counts
    turnstile_df_valuecounts["EXITS DIFFERENCE"] = (turnstile_df_merge12['EXITS_x'] - turnstile_df_merge12['EXITS_y']).abs() # Grabbing "EXITS" difference counts
    
    # Computes the "ENTRIES" + "EXITS" for each counter
    turnstile_df_valuecounts["TOTAL"] = turnstile_df_valuecounts["ENTRIES DIFFERENCE"] + turnstile_df_valuecounts["EXITS DIFFERENCE"]  
    
    # Grab names of "STATION" and number of times they appear in turnstile_df_valuecounts
    station_count = Counter(turnstile_df_valuecounts['STATION'])
    station_dict = {i:station_count[i] for i in station_count}
    
    # For each "STATION" name, sum up all the counts from all units
    value_count_dict = dict(zip(list(station_dict.keys()), [[turnstile_df_valuecounts.loc[turnstile_df_valuecounts['STATION'] == i, 'TOTAL'].sum()] for i in list(station_dict.keys())]))
    
    # Grab list of most popular stations and puts it into a sorted dataframe column
    value_count_df = pd.DataFrame.from_dict(value_count_dict).transpose() # Grabbing values
    value_count_df.columns = ["ValueCount"] # Renaming column to ValueCount
    value_count_df = value_count_df.sort_values(by=["ValueCount"], ascending=False) # Sorting by count
    
    return value_count_df

In [3]:
turnstile_csv_df = pd.read_csv(r"data/turnstile/downloads/turnstile_190330.txt")
turnstile_df = format_turnstile_df(turnstile_csv_df)
turnstile_df

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,TIME,DESC,ENTRIES,EXITS,DATETIMERAW
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,1900-01-01 00:00:00,REGULAR,6989774,2370411,2019-03-23 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,1900-01-01 04:00:00,REGULAR,6989795,2370413,2019-03-23 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,1900-01-01 08:00:00,REGULAR,6989813,2370436,2019-03-23 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,1900-01-01 12:00:00,REGULAR,6989924,2370512,2019-03-23 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,1900-01-01 16:00:00,REGULAR,6990200,2370573,2019-03-23 16:00:00
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,1900-01-01 20:00:00,REGULAR,6990562,2370623,2019-03-23 20:00:00
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,1900-01-01 00:00:00,REGULAR,6990734,2370648,2019-03-24 00:00:00
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,1900-01-01 04:00:00,REGULAR,6990758,2370653,2019-03-24 04:00:00
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,1900-01-01 08:00:00,REGULAR,6990772,2370676,2019-03-24 08:00:00
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,1900-01-01 12:00:00,REGULAR,6990860,2370731,2019-03-24 12:00:00


In [4]:
# Grab data between 8am and 12pm for the whole week
value_count_df = valuecount(dt.datetime(2019, 3, 23, 8, 0), dt.datetime(2019, 3, 23, 12, 0))
value_count_df_08to12 = value_count_df[["ValueCount"]]
value_count_df_08to12.columns = ["23"]

for i in range(24, 30):
    valuecount(dt.datetime(2019, 3, i, 8, 0), dt.datetime(2019, 3, i, 12, 0))
    value_count_df_08to12["{}".format(i)] = value_count_df[["ValueCount"]]

value_count_df_08to12.to_csv(r"data/value-count/valuecount_190330_df_08to12.csv")
value_count_df_08to12 # Preview

,23,24,25,26,27,28,29
34 ST-HERALD SQ,25176.0,25176.0,25176.0,25176.0,25176.0,25176.0,25176.0
34 ST-PENN STA,20202.0,20202.0,20202.0,20202.0,20202.0,20202.0,20202.0
TIMES SQ-42 ST,20062.0,20062.0,20062.0,20062.0,20062.0,20062.0,20062.0
FLUSHING-MAIN,15778.0,15778.0,15778.0,15778.0,15778.0,15778.0,15778.0
59 ST,14402.0,14402.0,14402.0,14402.0,14402.0,14402.0,14402.0
42 ST-PORT AUTH,14115.0,14115.0,14115.0,14115.0,14115.0,14115.0,14115.0
59 ST COLUMBUS,13942.0,13942.0,13942.0,13942.0,13942.0,13942.0,13942.0
86 ST,13641.0,13641.0,13641.0,13641.0,13641.0,13641.0,13641.0
GRD CNTRL-42 ST,13410.0,13410.0,13410.0,13410.0,13410.0,13410.0,13410.0
JKSN HT-ROOSVLT,13341.0,13341.0,13341.0,13341.0,13341.0,13341.0,13341.0


In [5]:
# Grab data between 12pm and 4pm for the whole week
value_count_df = valuecount(dt.datetime(2019, 3, 23, 12, 0), dt.datetime(2019, 3, 23, 16, 0))
value_count_df_12to16 = value_count_df[["ValueCount"]]
value_count_df_12to16.columns = ["23"]

for i in range(24, 30):
    valuecount(dt.datetime(2019, 3, i, 12, 0), dt.datetime(2019, 3, i, 16, 0))
    value_count_df_12to16["{}".format(i)] = value_count_df[["ValueCount"]]

value_count_df_12to16.to_csv(r"data/value-count/valuecount_190330_df_12to16.csv")
value_count_df_12to16 # Preview

,23,24,25,26,27,28,29
34 ST-HERALD SQ,40700.0,40700.0,40700.0,40700.0,40700.0,40700.0,40700.0
TIMES SQ-42 ST,34951.0,34951.0,34951.0,34951.0,34951.0,34951.0,34951.0
34 ST-PENN STA,31296.0,31296.0,31296.0,31296.0,31296.0,31296.0,31296.0
86 ST,23054.0,23054.0,23054.0,23054.0,23054.0,23054.0,23054.0
59 ST COLUMBUS,22925.0,22925.0,22925.0,22925.0,22925.0,22925.0,22925.0
59 ST,20674.0,20674.0,20674.0,20674.0,20674.0,20674.0,20674.0
GRD CNTRL-42 ST,20560.0,20560.0,20560.0,20560.0,20560.0,20560.0,20560.0
FLUSHING-MAIN,18083.0,18083.0,18083.0,18083.0,18083.0,18083.0,18083.0
CHAMBERS ST,17457.0,17457.0,17457.0,17457.0,17457.0,17457.0,17457.0
ATL AV-BARCLAY,16842.0,16842.0,16842.0,16842.0,16842.0,16842.0,16842.0


In [6]:
# Grab data between 4pm and 8pm for the whole week
value_count_df = valuecount(dt.datetime(2019, 3, 23, 16, 0), dt.datetime(2019, 3, 23, 20, 0))
value_count_df_16to20 = value_count_df[["ValueCount"]]
value_count_df_16to20.columns = ["23"]

for i in range(24, 30):
    valuecount(dt.datetime(2019, 3, i, 16, 0), dt.datetime(2019, 3, i, 20, 0))
    value_count_df_16to20["{}".format(i)] = value_count_df[["ValueCount"]]

value_count_df_16to20.to_csv(r"data/value-count/valuecount_190330_df_16to20.csv")
value_count_df_16to20 # Preview

,23,24,25,26,27,28,29
34 ST-HERALD SQ,47905.0,47905.0,47905.0,47905.0,47905.0,47905.0,47905.0
TIMES SQ-42 ST,36562.0,36562.0,36562.0,36562.0,36562.0,36562.0,36562.0
34 ST-PENN STA,33898.0,33898.0,33898.0,33898.0,33898.0,33898.0,33898.0
59 ST COLUMBUS,24166.0,24166.0,24166.0,24166.0,24166.0,24166.0,24166.0
86 ST,22782.0,22782.0,22782.0,22782.0,22782.0,22782.0,22782.0
GRD CNTRL-42 ST,20555.0,20555.0,20555.0,20555.0,20555.0,20555.0,20555.0
59 ST,20197.0,20197.0,20197.0,20197.0,20197.0,20197.0,20197.0
FLUSHING-MAIN,19467.0,19467.0,19467.0,19467.0,19467.0,19467.0,19467.0
ATL AV-BARCLAY,17247.0,17247.0,17247.0,17247.0,17247.0,17247.0,17247.0
23 ST,17217.0,17217.0,17217.0,17217.0,17217.0,17217.0,17217.0
